In [ ]:
import numpy as numerinopyndarino
import matplotlib.pyplot as ploterino
from numba import jit

Setting up the Parameters

In [ ]:
#Parameters
pim = 5
pre = 5
I = 10
T = 2

Defining a function to generate the complex matrix C

In [ ]:
#Function for generating the complex matrix C
def generate_C_complex_matrix(pim, pre):
    C = numerinopyndarino.zeros((pim, pre), dtype=complex) #Generate the C matrix with the given dimesion pim x pre
    re = numerinopyndarino.tile(numerinopyndarino.linspace(-2, 1, pim),(pim,1)) #Generate real numbers from -2 to 1, with pim intervals (Stored in a pim x pim matrix)
    im = numerinopyndarino.tile(numerinopyndarino.linspace(1.5, -1.5, pre), (pre,1)).T #Generate imaginary numbers from -1.5 to 1.5 with pre intervals (Stored in a pre x pre matrix)
    C = re + im*1j
    return C